In [1]:
import logging
import sys
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext, load_index_from_storage
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
#create index for all files
#total of 18 files from azure arm template repo: https://github.com/Azure/azure-quickstart-templates
documents = SimpleDirectoryReader("data").load_data()

# bge embedding model
# no custom parameters has been passed to embedding model
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

index = VectorStoreIndex.from_documents(
    documents,
)
index.storage_context.persist(persist_dir="./indexes")

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 18
> [SimpleDirectoryReader] Total files added: 18
DEBUG:fsspec.local:open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azure-storage-metadata.json
open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azure-storage-metadata.json
DEBUG:fsspec.local:open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azuredeploy-keyvault.json
open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azuredeploy-keyvault.json
DEBUG:fsspec.local:open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azuredeploy-keyvault.parameters.json
open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azuredeploy-keyvault.parameters.json
DEBUG:fsspec.local:open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azuredeploy-kv-keys.json
open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azuredeploy-kv-keys.json
DEBUG:fsspec.local:

In [3]:
# ollama
Settings.llm = Ollama(model="mistral", request_timeout=300.0)
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./indexes")
# load index
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine()
response = query_engine.query("Give me a JSON arm template to deploy a storage account and a virtual machine. Virtual machine should be able to access storage account.")
print(response)

DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ./indexes/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./indexes/docstore.json.
DEBUG:fsspec.local:open file: /Users/vishwajeetkumar/work/arm-template-generator/indexes/docstore.json
open file: /Users/vishwajeetkumar/work/arm-template-generator/indexes/docstore.json
DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ./indexes/index_store.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./indexes/index_store.json.
DEBUG:fsspec.local:open file: /Users/vishwajeetkumar/work/arm-template-generator/indexes/index_store.json
open file: /Users/vishwajeetkumar/work/arm-template-generator/indexes/index_store.json
DEBUG:llama_index.core.graph_stores.simple:Loading llama_index.core.graph_stores.simple from ./indexes/graph_store.json.
Loading llama_index.core.graph_stores.sim